In [113]:
import json
import glob
import os
import jieba
import re
# AllDocRoot 為 所有評論所在資料夾
AllDocRoot = r"C:\Users\User\Desktop\商管\Final\TenReviews"
#CP值的csv轉成utf-8 的txt所在位置
CPPath = r'C:\Users\User\Documents\GitHub\2019fall_final\CP.txt'
EnvironmentPath = r'C:\Users\User\Documents\GitHub\2019fall_final\環境.txt'
ServicePath = r'C:\Users\User\Documents\GitHub\2019fall_final\服務.txt'
DelicacyPath = r'C:\Users\User\Documents\GitHub\2019fall_final\美味.txt'
#  務必下載斷詞字典和停用詞字典放到以下位置
SepDictionaryPath = r'C:\Users\User\AppData\Local\Programs\Python\Python37\Lib\jieba\dict.txt'
StopDicPath = r'C:\Users\User\AppData\Local\Programs\Python\Python37\Lib\jieba\stop.txt'
jieba.set_dictionary(r'C:\Users\User\AppData\Local\Programs\Python\Python37\Lib\jieba\dict.txt')

with open(StopDicPath,'r',encoding = 'utf8') as f:
    StopWords = f.read().split('\n')
    StopWords[0] = ','
    
def ChiFeatureSelection(Corpus_list,dic_list,Class_group, n ):
    '''
    Feautures Selection的方法
    Input:Corpus_list, [0] 為標記 0,1 [1]為斷詞後評論
          dic_list, set of terms
          Class_group, 共幾組，以list表示 i.e. ['0','1']
          n, 前幾名term
    '''
    goodfeatures = []
    for term in dic_list:
        Xchi = 0
        ContingencyTable = []
        Tcount= 0
        Fcount = 0
        for Class in Class_group:
            TP = 0
            FN = 0
            for corpus in Corpus_list:
                
                if term in corpus[1] and Class == corpus[0]:
                    TP += 1
                elif term not in corpus[1] and Class == corpus[0]:
                    FN += 1
            Tcount += TP
            Fcount += FN
            ContingencyTable.append([TP,FN])
#             print(Class)
            
        Total = (Tcount + Fcount) 
        T_P = Tcount / Total
        N_P = 1 - T_P
#         print(T_P,N_P,sep=' ')
        for i in ContingencyTable:
            T = i[0]
            F = i[1]
            Class_P = (T + F)/ Total
            
            Etc = T_P * Class_P
            Efc = N_P * Class_P
            if N_P ==0:
                print(N_P )
            Xchi += (T - Etc)**(2) / Etc + (F - Efc)**(2) / Efc
        goodfeatures.append([term,Xchi])
        goodfeatures.sort(reverse = True, key = lambda x:x[1])
        goodfeatures_term = []
        for i in goodfeatures:
            goodfeatures_term.append(i[0])
    return goodfeatures_term[0:n]
    

In [130]:
class Training:
    import jieba
    
    def __init__(self,Path):
        '''
        Input: Path, 為訓練集的位置的，UTF-8 txt
        Output: Positive, 正面(1)或負面(0)
                Name, 店家名
                Counter ,該店家第幾篇評論
        '''
        Corpus = []
        with open(Path,'r', encoding = 'utf-8') as text:
        #     text.close()
            PositiveList = []
            NameList = []
            CounterList = []
            for i in text:
                Positive = i.strip('\n').split('\t')[0]
                Positive = Positive.strip('\ufeff')
                Name = i.strip('\n').split('\t')[1].split(',')[0]
                Id =int(i.strip('\n').split('\t')[1].split(',')[1])
                PositiveList.append(Positive)
                NameList.append(Name)
                CounterList.append(Id)
                
        self.Positive = PositiveList
        self.Name= NameList
        self.Counter = CounterList
        
    def BuildTrainDic(self,ALLDocsRoot,n):
        '''
        Input:  ALLDocsRoot, 為所有評論的位置，評論為json檔
                Positive, 正面(1)或負面(0)
                Name, 店家名
                Counter ,該店家第幾篇評論
                皆為上述產生
                n, 前n名的Feautures
        Ootput:
                self.TopNFeautures,前n名的Feautures
                
        41~70 對你找出資料夾中訓練集的文章並清掉韓文有幫助，71是結疤斷詞
        '''
        file_list = glob.glob(os.path.join(os.getcwd(), ALLDocsRoot, "*.json"))
        Corpus = []
        NegativeCorpus = []
        NameAndPositive = []
        Potential_Features = []
        Potential_GFeatures = []
        Potential_NFeatures = []
        for i in zip(self.Positive,self.Name):
            NameAndPositive.append([i[1],i[0]])
       
        for file_path in file_list:
            BaseName = file_path.split('\\')[-1].strip('.json')
            for NamePositiveAndOrder in zip(NameAndPositive,self.Counter):
#                 print(NamePositiveAndOrder[0][0],BaseName)
                if BaseName == NamePositiveAndOrder[0][0] :                   
                    with open(file_path,encoding = 'utf-8') as load_test:
                        doc =json.load(load_test)
                        
                        if len(doc['reviews']) != 0:
                            
                            for i in range(len(doc['reviews'])):
                                if i == NamePositiveAndOrder[1]:
                                    
                                    Text = doc['reviews'][i]['text']
                                    Text = re.findall(r'[\u4e00-\u9fff]+', Text)
                                    Sisstring = ''
                                    for i in Text:
                                        Sisstring += i 
                                    words = [s for s in jieba.cut(Sisstring, cut_all=False) if s not in StopWords]
                                    if NamePositiveAndOrder[0][1] == '1':
                                        Corpus.append([NamePositiveAndOrder[0][1],words])
                                        Potential_GFeatures.extend(words)
                                        Potential_Features.extend(words)
                                    elif NamePositiveAndOrder[0][1] == '0':
                                        Corpus.append([NamePositiveAndOrder[0][1],words])
                                        Potential_NFeatures.extend(words)
                                        Potential_Features.extend(words)
                                    else:
                                        return print('Indicatior is neither 1 or 0')

        self.Potential_Features = set(Potential_Features)
        self.GoodFeatures = set(Potential_GFeatures)
        self.NegativeFeatures = set(Potential_NFeatures)
        
        Class_group= ['0','1']
        
        self.TopNFeautures = ChiFeatureSelection(Corpus, self.Potential_Features,Class_group, n)
        self.TopNGood = ChiFeatureSelection(Corpus, self.GoodFeatures,Class_group, n)
        self.TopNNegative = ChiFeatureSelection(Corpus, self.NegativeFeatures,Class_group, n)
        
#                 CorpusPerReviewTemp = []
#                 CorpusPerVendorTemp= []
#                 AvgRate = 0
#                 Rate = None
#                 if len(doc['reviews']) == 0:
#                     for i in range(len(doc['reviews'])):

#                         if CounterForSampling in SampleList:

#                             Rate = float(doc['reviews'][i]['rating'])
#                             AvgRate += Rate
#                             TextRaw = doc['reviews'][i]['text']

#                             if '(원본)' in TextRaw :
#                                 TextChinese = TextRaw.split('(원본)')[-1]
#                             else:
#                                 TextChinese = TextRaw

#                             TextChineseRemoveN = TextChinese.replace('\n','')
#                             CorpusPerReviewTemp.append([Rate,TextChineseRemoveN])
#                             CorpusPerVendorTemp.append(TextChineseRemoveN)

#                              CounterForSampling +=1

#                     AvgRate = AvgRate / len(doc['reviews'])     
#                     CorpusPerVendor[BaseName] = [AvgRate,CorpusPerVendorTemp] 
#                     CorpusPerReview[BaseName] = "".join(CorpusPerVendorTemp)
#                     AllCorpus.extend(CorpusPerVendorTemp)


#                 else:
#                     pass
#         AllCorpus = ".".join(AllCorpus)
        
        

In [124]:
CPPath = r'C:\Users\User\Documents\GitHub\2019fall_final\CP.txt'
EnvironmentPath = r'C:\Users\User\Documents\GitHub\2019fall_final\環境.txt'
ServicePath = r'C:\Users\User\Documents\GitHub\2019fall_final\服務.txt'
DelicacyPath = r'C:\Users\User\Documents\GitHub\2019fall_final\美味.txt'

CP = Training(CPPath)
Environment = Training(EnvironmentPath)
Service = Training(ServicePath)
Delicacy =Training(DelicacyPath)


In [132]:
Training.BuildTrainDic(CP,AllDocRoot, 60)
# Training.BuildTrainDic(Environment ,AllDocRoot, 60)
# Training.BuildTrainDic(Service ,AllDocRoot, 60)
# Training.BuildTrainDic(Delicacy ,AllDocRoot, 60)

print(CP.TopNGood)
print(CP.TopNNegative)
print(CP.TopNFeautures)
# print(Environment.TopNFeautures)
# print(Service.TopNFeautures)
# print(Delicacy.TopNFeautures)

['老闆', '味道', '便宜', '划算', '店', '聚餐', '值高', '人', '台大', '品質', '算', '一起', '不錯', '平價', '豆腐', '上菜', '免費', '主菜', '挺', '值得', '喝到', '必點', '時', '試試', '機會', '搭配', '咖哩', '食材', '老闆娘', '白飯', '清爽', '包', '點餐', '不貴', '特別', '一個', '乾淨', '評價', '不飽', '主餐', '醬汁', '少', '裝潢', '小菜', '鹹', '適合', '喜歡', '親切', '比較', '價位', '有點', '好吃', '飯', '再', '馬鈴薯', '無限', '分', '簡單', '推', '柴魚片']
['普通', '應該', '價錢', '附近', '加點', '知道', '老闆', '味道', '人', '台大', '品質', '算', '兩顆', '久坐', '青菜', '再來', '接受', '聊天', '還要', '這次', '好了', '趕人', '算是', '分量', '配菜', '不差', '越來越', '要價', '左右', '為主', '便當', '關係', '門口', '卻', '單點', '貴', '最近', '名單', '三顆', '之前', '不錯', '平價', '特別', '一個', '乾淨', '評價', '不飽', '主餐', '醬汁', '少', '裝潢', '小菜', '鹹', '適合', '喜歡', '親切', '比較', '價位', '有點', '好吃']
['普通', '應該', '價錢', '加點', '附近', '知道', '老闆', '味道', '便宜', '聚餐', '划算', '店', '值高', '人', '算', '台大', '品質', '一起', '久坐', '這次', '門口', '名單', '青菜', '接受', '聊天', '還要', '好了', '算是', '越來越', '左右', '便當', '關係', '最近', '再來', '不差', '要價', '為主', '卻', '單點', '兩顆', '趕人', '分量', '配菜', '貴', '三顆', '之前', '不錯', '平價', '豆腐', '